In [ ]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
x = torch.linspace(0, 799, 800)
y = torch.sin(x*2*3.1416/40)

In [ ]:
plt.figure(figsize=(15,5))
plt.xlim(-10, 810)
plt.grid(True)
plt.plot(y.numpy())

In [ ]:
# here we split the data to train and test section
test_size = 40
train_set = y[:-test_size]
test_set = y[-test_size:]

In [ ]:
plt.figure(figsize=(15,5))
plt.xlim(-10, 810)
plt.grid(True)
plt.plot(train_set.numpy())

In [ ]:
# here we divide the training data to sequences that can be feeded to the network
def input_data(data_seq, window_size):
    out = []      # 'out' would be like this : [([0,1,2,3],[4]), ([1,2,3,4],[5]), ([2,3,4,5],[6]), ...]
    
    for i in range(len(data_seq)-window_size):
        window = data_seq[i:i+window_size]
        label = data_seq[i+window_size:i+window_size+1]
        out.append((window, label))
    
    return out

In [ ]:
window_size = 40
train_data = input_data(train_set, window_size)

In [ ]:
len(train_data)

In [ ]:
train_data[0]

In [ ]:
train_data[1]

In [ ]:
class ModelLSTM(nn.Module):
    # "input_size" is 1 because we have one 'y' value per timestamp
    # "hidden_size" is the number lstm neurons inside the LSTM layer which is located before the fully connected layer
    # "out_size" is 1 becuase we produce one value as label
    def __init__(self, input_size=1, hidden_size=50, out_size=1):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size)
        
        self.linear = nn.Linear(hidden_size, out_size)
        
        # placeholder for hidden-state and cell-state
        self.hidden_state_cell_state = (torch.zeros(1,1,hidden_size), torch.zeros(1,1,hidden_size))
        
    def forward(self, data_seq):
        lstm_out, self.hidden_state_cell_state = self.lstm(data_seq.view(len(data_seq),1,-1),self.hidden_state_cell_state)
        pred = self.linear(lstm_out.view(len(data_seq), -1)[-1])
        
        return pred

In [ ]:
torch.manual_seed(42)
model = ModelLSTM()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
model

In [ ]:
for param in model.parameters():
    print(param.numel())

In [ ]:
epochs = 10
future_points = 40

for i in range(epochs):
    for data_seq, y_train in train_data:
        # first have to reset the parameters and hidden states
        optimizer.zero_grad()
        model.hidden_state_cell_state = (torch.zeros(1,1,model.hidden_size),torch.zeros(1,1,model.hidden_size))
        
        y_pred = model.forward(data_seq)
        
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch:{i} loss:{loss.item()}")
    
    preds = train_set[-window_size:].tolist()
    for f in range(future_points):
        seq = torch.FloatTensor(preds[-window_size:])
        
        with torch.no_grad():
            model.hidden_state_cell_state = (torch.zeros(1,1,model.hidden_size),torch.zeros(1,1,model.hidden_size))
            preds.append(model.forward(seq).item())
    loss = criterion(torch.tensor(preds[-window_size:]), y[760:])
    print(f"Performance on the test range; the loss: {loss}")
    
    plt.figure(figsize=(12,4))
    plt.xlim(700,801)
    plt.grid(True)
    plt.plot(y.numpy())
    plt.plot(range(760,800), preds[window_size:])
    plt.show()

In [ ]:
# now we want to use all the possible data for training and use the last section to 'forcast into unknown future':
epochs = 15
window_size = 40
future = 40

all_data = input_data(y, window_size)
len(all_data)

In [ ]:
import time
start_time = time.time()

for i in range(epochs):
    for data_seq, y_train in all_data:
        # first have to reset the parameters and hidden states
        optimizer.zero_grad()
        model.hidden_state_cell_state = (torch.zeros(1,1,model.hidden_size),torch.zeros(1,1,model.hidden_size))
        
        y_pred = model.forward(data_seq)
        
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch:{i} loss:{loss.item()}")

total_time = time.time()-start_time
print(f'The training time is: {total_time/60} mins')

In [ ]:
# now use the trained model for forcasting:
preds = y[-window_size:].tolist()
for f in range(future):
    seq = torch.FloatTensor(preds[-window_size:])

    with torch.no_grad():
        model.hidden_state_cell_state = (torch.zeros(1,1,model.hidden_size),torch.zeros(1,1,model.hidden_size))
        preds.append(model.forward(seq).item())

plt.figure(figsize=(12,4))
plt.xlim(0,841)
plt.grid(True)
plt.plot(y.numpy())
plt.plot(range(800,800+future), preds[window_size:])
plt.show()